In [1]:
!tar -czvf ./sourcedir.tar.gz inference.py requirements.txt utils.py

inference.py
requirements.txt
utils.py


In [237]:
#use model from s3 directly
#!tar -czvf ../model.tar.gz /home/ec2-user/SageMaker/shopplazza/sam_vit_h_4b8939.pth

In [2]:
#note: need to change s3 location
!aws s3 cp ./sourcedir.tar.gz s3://fashionai2023/sam/
#!aws s3 cp ../model.tar.gz s3://fashionai2023/sam/

upload: ./sourcedir.tar.gz to s3://fashionai2023/sam/sourcedir.tar.gz


In [5]:
import sagemaker
from sagemaker.pytorch import PyTorch
import matplotlib.pyplot as plt

In [6]:
import boto3
from sagemaker import get_execution_role

role = get_execution_role()
sagemaker_session = sagemaker.Session()
bucket = sagemaker_session.default_bucket()

#role = "arn:aws:iam::344510699720:role/sagemaker_execution_role"

In [7]:
model_name = 'sam-model-v1'
entry_point = 'inference.py'
framework_version = '1.10.2'
py_version = 'py38'

# model_type: sam_vit_b, sam_vit_l, sam_vit_h
model_environment = {
    'model_type': 'sam_vit_h',
    'TS_MAX_RESPONSE_SIZE':'2000000000',
    'TS_MAX_REQUEST_SIZE':'2000000000',
    'MMS_MAX_RESPONSE_SIZE':'2000000000',
    'MMS_MAX_REQUEST_SIZE':'2000000000'
}

In [8]:
source_dir = 's3://fashionai2023/sam/sourcedir.tar.gz'
model_data = 's3://fashionai2023/sam/model.tar.gz'

In [9]:
!aws s3 ls s3://fashionai2023/sam/

2023-09-05 07:40:01 2380812814 model.tar.gz
2023-09-19 06:47:20       4048 sourcedir.tar.gz


In [10]:
from sagemaker.pytorch.model import PyTorchModel

model = PyTorchModel(
    name = model_name,
    model_data = model_data,
    entry_point = entry_point,
    source_dir = source_dir,
    role = role,
    framework_version = framework_version, 
    py_version = py_version,
    env = model_environment
)

In [11]:
endpoint_name = 'sam-endpoint-v1'
instance_type = 'ml.g5.xlarge'
instance_count = 1

In [12]:
%%time

#10min around
predictor = model.deploy(
    endpoint_name = endpoint_name,
    instance_type = instance_type, 
    initial_instance_count = instance_count
)

---------------!CPU times: user 2min 26s, sys: 31.1 s, total: 2min 57s
Wall time: 10min 34s


# invoke

In [15]:

from boto3.session import Session
import json
import base64

#change test image path if needed 
INPUT_IMAGE = '/home/ec2-user/SageMaker/shoplazza_optimize/test.jpeg'

with open(INPUT_IMAGE,"rb") as f:
    body_base64 = base64.b64encode(f.read()).decode()
    

In [ ]:
%%time
input_data = {
        "body": body_base64,
        "task": "AutomaticMaskGenerator",
        "points_per_side":32,
        "points_per_batch":128,
        "pred_iou_thresh":0.88,
        "stability_score_thresh":0.9,
        "stability_score_offset":1.0,
        "box_nms_thresh":0.95,
        "crop_n_layers":0,
        "crop_nms_thresh":0.7,
        "crop_overlap_ratio":0.3413333333333333,
        "crop_n_points_downscale_factor":1,
        "min_mask_region_area":0,
        }

session = Session()
runtime = session.client("runtime.sagemaker")

response = runtime.invoke_endpoint(
    EndpointName=endpoint_name,
    ContentType="image/jpg",
    Body=json.dumps(input_data),
)


In [ ]:
response['Body'].read()

# gsam

In [252]:

from boto3.session import Session
import json
import base64


#change test image path if needed 
INPUT_IMAGE = '/home/ec2-user/SageMaker/shoplazza_optimize/test.jpeg'

with open(INPUT_IMAGE,"rb") as f:
    body_base64 = base64.b64encode(f.read()).decode()
    

In [253]:
%%time
input_data = {
        "body": body_base64,
        "task": "gsam",
        "box_threshold": 0.1,
        "text_threshold": 0.1,
        "text_prompt": "person"
        }

session = Session()
runtime = session.client("runtime.sagemaker")

response = runtime.invoke_endpoint(
    EndpointName=endpoint_name,
    ContentType="image/jpg",
    Body=json.dumps(input_data),
)


CPU times: user 49.5 ms, sys: 6.3 ms, total: 55.8 ms
Wall time: 739 ms


In [254]:
response['Body'].read()

b'{"result": {"task": "gsam", "masks": "LoVQWgjAJhIHwCMAMABAvAYQJwAldYHEB2AKQDZySiBJMgaQBYAZBuspgZiYFY6AOagwDGdAKYkANgFkA+twC2EcQCUAcgANuAM2gkwqgHrd1ELSRmq5ADVMkA5hu4AFXU4Dymu0nFW3AQ244KBIAIQDuIRI6KG4SdRJuGIAReIAeGIBtfxJRJCFuZTcEEnkkBG4AC1UzAHUII30AOxIEaCkwJj8iAEsIAGU6AGsIGoUpeRIARQAmAE1tKQkSK2mM4ykK82ngbhqpbpIK6YAdbkmpOxJG6blBpkiAFQ5RRKZjkmOOB6w3CF-fqQcAEcVTTUEsFjUIgsUGg-5-CBwxHTeFIJCo9EI36w6aApF4sGE7FE5HE+FI+GUiBojG-GkItH0pmYxks1mMuns5lcnm8vl-VkCpBYtEigD2AFE+AArDjUEBMABcFQ4BAAakxUiqJTUmGhBnQHkUyAB+IRSE2iOizNyiMhYKBSdTeOhgW1kMhaKTIPx0Y5uOxkDheqBIdR+twVMgQOxSXLAOgGNzSsiqCpSLRIWZ0ACC8go3X2SAGOb6RBw6ZmNWmTjTUirNbrDcmqljzdbUic0xWalEnemYGmfVU5saEGO0weqj8UjHBmmIEsiwg6mmNVUxykwyE3dUsyk0tMA9UD32EEG03UG5IfjTSEmHEGDC0qgk1CgqhAEG600aDEmbhgNQxzVEef4MM0biTNwHDSiQsDcEgly7NwUgZDeeTcDmNQkKGFTcDIUEkJmjTcMAbhvhUSDKGQUAMG4YJ-IxDGwkxqhsWxhKqBQvAUGxHCAoJAnMMwvC8VQ1B8HgBAANwSoqABiqRJIpylqUkymKiEaooNMYoyA8EBAA"}}'

# endpoint delete

In [235]:
import boto3

endpoint_name = 'sam-model-v1'
model_name = 'sam-endpoint-v1'
# Get the SageMaker client
#sagemaker_client = boto3.client("sagemaker")

# Delete the endpoint
sagemaker_client.delete_endpoint(EndpointName=endpoint_name)

# Delete the endpoint configuration
sagemaker_client.delete_endpoint_config(EndpointConfigName=endpoint_name)

# Delete the model
sagemaker_client.delete_model(ModelName=model_name)


{'ResponseMetadata': {'RequestId': '60ee8d68-417f-4488-8014-afc20841f8a1',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '60ee8d68-417f-4488-8014-afc20841f8a1',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '0',
   'date': 'Wed, 06 Sep 2023 06:51:44 GMT'},
  'RetryAttempts': 0}}